# **<font color=white> 11.Regularized Model-ElasticNet Code 실습**

[목적]
  1. ElasticNet
    - Regularized Linear Model을 활용하여 Overfitting을 방지함
    - Hyperparameter lamba를 튜닝할 때 for loop 뿐만 아니라 GridsearchCV를 통해 돌출해봄
  3. Regularized Linear Models의 경우 X's Scaling을 필수적으로 진행해야함

[Process]
  1. Define X's & Y
  2. Split Train & Valid dataset
  3. Modeling
  4. Model 해석

In [1]:
# Package
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet, ElasticNetCV
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

# Sklearn Toy Data
from sklearn.datasets import load_diabetes

In [2]:
# Data Loading (당뇨병)
data = pd.read_csv('https://www4.stat.ncsu.edu/~boos/var.select/diabetes.tab.txt', sep='\t')

In [3]:
# X's & Y Split
Y = data['Y']
X = data.drop(columns=['Y']) 
X = pd.get_dummies(X, columns=['SEX'])

[Data Split]

- Data Split을 진행할 때 BigData의 경우 꼭 indexing을 추출하여 모델에 적용시켜야 함
- 이유는 Data Split하여 새로운 Data set을 만들 경우 메모리에 부담을 주기 때문

In [4]:
idx = list(range(X.shape[0]))
train_idx, valid_idx = train_test_split(idx, test_size=0.3, random_state=2023)
print(">>>> # of Train data : {}".format(len(train_idx)))
print(">>>> # of valid data : {}".format(len(valid_idx)))

>>>> # of Train data : 309
>>>> # of valid data : 133


In [5]:
# Scaling
scaler = MinMaxScaler().fit(X.iloc[train_idx])
X_scal = scaler.transform(X)
X_scal = pd.DataFrame(X_scal, columns=X.columns)

In [6]:
import scipy
from sklearn import metrics

def sse(clf, X, y):
    """Calculate the standard squared error of the model.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    float
        The standard squared error of the model.
    """
    y_hat = clf.predict(X)
    sse = np.sum((y_hat - y) ** 2)
    return sse / X.shape[0]


def adj_r2_score(clf, X, y):
    """Calculate the adjusted :math:`R^2` of the model.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    float
        The adjusted :math:`R^2` of the model.
    """
    n = X.shape[0]  # Number of observations
    p = X.shape[1]  # Number of features
    r_squared = metrics.r2_score(y, clf.predict(X))
    return 1 - (1 - r_squared) * ((n - 1) / (n - p - 1))


def coef_se(clf, X, y):
    """Calculate standard error for beta coefficients.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    numpy.ndarray
        An array of standard errors for the beta coefficients.
    """
    n = X.shape[0]
    X1 = np.hstack((np.ones((n, 1)), np.matrix(X)))
    se_matrix = scipy.linalg.sqrtm(
        metrics.mean_squared_error(y, clf.predict(X)) *
        np.linalg.inv(X1.T * X1)
    )
    return np.diagonal(se_matrix)


def coef_tval(clf, X, y):
    """Calculate t-statistic for beta coefficients.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    numpy.ndarray
        An array of t-statistic values.
    """
    a = np.array(clf.intercept_ / coef_se(clf, X, y)[0])
    b = np.array(clf.coef_ / coef_se(clf, X, y)[1:])
    return np.append(a, b)


def coef_pval(clf, X, y):
    """Calculate p-values for beta coefficients.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    Returns
    -------
    numpy.ndarray
        An array of p-values.
    """
    n = X.shape[0]
    t = coef_tval(clf, X, y)
    p = 2 * (1 - scipy.stats.t.cdf(abs(t), n - 1))
    return p

def summary(clf, X, y, xlabels=None):
    """
    Output summary statistics for a fitted regression model.
    Parameters
    ----------
    clf : sklearn.linear_model
        A scikit-learn linear model classifier with a `predict()` method.
    X : numpy.ndarray
        Training data used to fit the classifier.
    y : numpy.ndarray
        Target training values, of shape = [n_samples].
    xlabels : list, tuple
        The labels for the predictors.
    """
    # Check and/or make xlabels
    ncols = X.shape[1]
    if xlabels is None:
        xlabels = np.array(
            ['x{0}'.format(i) for i in range(1, ncols + 1)], dtype='str')
    elif isinstance(xlabels, (tuple, list)):
        xlabels = np.array(xlabels, dtype='str')
    # Make sure dims of xlabels matches dims of X
    if xlabels.shape[0] != ncols:
        raise AssertionError(
            "Dimension of xlabels {0} does not match "
            "X {1}.".format(xlabels.shape, X.shape))
    # Create data frame of coefficient estimates and associated stats
    coef_df = pd.DataFrame(
        index=['_intercept'] + list(xlabels),
        columns=['Estimate', 'Std. Error', 't value', 'p value']
    )
    try:
        coef_df['Estimate'] = np.concatenate(
            (np.round(np.array([clf.intercept_]), 6), np.round((clf.coef_), 6)))
    except Exception as e:
        coef_df['Estimate'] = np.concatenate(
            (
                np.round(np.array([clf.intercept_]), 6),
                np.round((clf.coef_), 6)
            ), axis = 1
    )[0,:]
    coef_df['Std. Error'] = np.round(coef_se(clf, X, y), 6)
    coef_df['t value'] = np.round(coef_tval(clf, X, y), 4)
    coef_df['p value'] = np.round(coef_pval(clf, X, y), 6)
    # Output results
    print('Coefficients:')
    print(coef_df.to_string(index=True))
    print('---')
    print('R-squared:  {0:.6f},    Adjusted R-squared:  {1:.6f},    MSE: {2:.1f}'.format(
        metrics.r2_score(y, clf.predict(X)), adj_r2_score(clf, X, y), sse(clf, X, y)))

[ElasticNet Regression]

  - Hyperparameter Tuning using for Loop
  - Hyperparameter Tuning using GridSearchCV

[ElasticNet Regression Parameters]
  - Package : https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html
  - alpha : L2-norm Penalty Term
    - alpha : 0 일 때, Just Linear Regression
  - l1_ratio : L1-norm Penalty Term
    - 0 <= l1_ratio <= 1
    - l1_ratio : 1 일 때, Just Ridge Regression
  - fit_intercept : Centering to zero
    - 베타0를 0로 보내는 것 (베타0는 상수이기 때문에)
  - max_iter : Maximum number of interation
    - Loss Function의 LASSO Penalty Term은 절대 값이기 때문에 Gradient Descent와 같은 최적화가 필요함
  - Penalty Term 
    - 1 / (2 * n_samples) * ||y - Xw||^2_2 + <u>alpha * l1_ratio * ||w||_1</u> + <u>0.5 * alpha * (1 - l1_ratio) * ||w||^2_2</u>

In [7]:
# alphas, alpha = 0 is equivalent to an ordinary least square, solved by the LinearRegression object.
alphas = [0.000001, 0.000005, 0.00001, 0.00005, 0.0001, 0.001, 0.005, 0.01, 0.05]
# betas range (0 ~ 1), l1_ratio is often to put more values close to 1 (i.e. Lasso) and less close to 0 (i.e. Ridge)
l1_ratio = [0.9, 0.7, 0.5, 0.3, 0.1]

# ElasticNet Regression
# select alpha and beta by checking R2, MSE, RMSE
for a in alphas:
    for b in l1_ratio:
        model = ElasticNet(alpha=a, l1_ratio=b).fit(X_scal.iloc[train_idx], Y.iloc[train_idx]) 
        score = model.score(X_scal.iloc[valid_idx], Y.iloc[valid_idx])
        pred_y = model.predict(X_scal.iloc[valid_idx])
        mse = mean_squared_error(Y.iloc[valid_idx], pred_y)
        print("Alpha:{0:.7f}, l1_ratio: {1:.7f}, R2:{2:.7f}, MSE:{3:.7f}, RMSE:{4:.7f}".format(a, b, score, mse, np.sqrt(mse)))

Alpha:0.0000010, l1_ratio: 0.9000000, R2:0.5301664, MSE:3084.6036665, RMSE:55.5392084
Alpha:0.0000010, l1_ratio: 0.7000000, R2:0.5301689, MSE:3084.5871229, RMSE:55.5390594
Alpha:0.0000010, l1_ratio: 0.5000000, R2:0.5301714, MSE:3084.5706467, RMSE:55.5389111
Alpha:0.0000010, l1_ratio: 0.3000000, R2:0.5301739, MSE:3084.5542376, RMSE:55.5387634
Alpha:0.0000010, l1_ratio: 0.1000000, R2:0.5301764, MSE:3084.5378953, RMSE:55.5386163
Alpha:0.0000050, l1_ratio: 0.9000000, R2:0.5301716, MSE:3084.5694666, RMSE:55.5389005
Alpha:0.0000050, l1_ratio: 0.7000000, R2:0.5301840, MSE:3084.4881189, RMSE:55.5381681


Alpha:0.0000050, l1_ratio: 0.5000000, R2:0.5301962, MSE:3084.4084152, RMSE:55.5374506
Alpha:0.0000050, l1_ratio: 0.3000000, R2:0.5302081, MSE:3084.3303231, RMSE:55.5367475
Alpha:0.0000050, l1_ratio: 0.1000000, R2:0.5302197, MSE:3084.2538107, RMSE:55.5360587
Alpha:0.0000100, l1_ratio: 0.9000000, R2:0.5301781, MSE:3084.5271311, RMSE:55.5385193
Alpha:0.0000100, l1_ratio: 0.7000000, R2:0.5302023, MSE:3084.3677950, RMSE:55.5370849
Alpha:0.0000100, l1_ratio: 0.5000000, R2:0.5302256, MSE:3084.2148395, RMSE:55.5357078
Alpha:0.0000100, l1_ratio: 0.3000000, R2:0.5302480, MSE:3084.0680163, RMSE:55.5343859
Alpha:0.0000100, l1_ratio: 0.1000000, R2:0.5302695, MSE:3083.9270883, RMSE:55.5331170
Alpha:0.0000500, l1_ratio: 0.9000000, R2:0.5302272, MSE:3084.2044920, RMSE:55.5356146
Alpha:0.0000500, l1_ratio: 0.7000000, R2:0.5303300, MSE:3083.5299323, RMSE:55.5295411
Alpha:0.0000500, l1_ratio: 0.5000000, R2:0.5304135, MSE:3082.9815630, RMSE:55.5246032
Alpha:0.0000500, l1_ratio: 0.3000000, R2:0.5304811, MS

In [8]:
# Cross Validation for ElasticNet
grid = dict()
grid['alpha'] = alphas
grid['l1_ratio'] = l1_ratio

In [9]:
grid

{'alpha': [1e-06, 5e-06, 1e-05, 5e-05, 0.0001, 0.001, 0.005, 0.01, 0.05],
 'l1_ratio': [0.9, 0.7, 0.5, 0.3, 0.1]}

In [10]:
# define model
model = ElasticNet()
# define search
search = GridSearchCV(model, grid, scoring='neg_root_mean_squared_error', cv=5, n_jobs=-1)
# perform the search
results = search.fit(X_scal.iloc[valid_idx], Y.iloc[valid_idx])
# summarize
print('RMSE: {:.4f}'.format(-results.best_score_))
print('Config: {}'.format(results.best_params_))

RMSE: 56.8906
Config: {'alpha': 0.01, 'l1_ratio': 0.3}


In [11]:
model_best = ElasticNet(alpha=results.best_params_['alpha'], 
                        l1_ratio=results.best_params_['l1_ratio']).fit(X_scal.iloc[train_idx], Y.iloc[train_idx])
score = model_best.score(X_scal.iloc[valid_idx], Y.iloc[valid_idx])
pred_y = model_best.predict(X_scal.iloc[valid_idx])
mse = mean_squared_error(Y.iloc[valid_idx], pred_y)
print("Alpha:{0:.7f}, l1_ratio: {1:.7f}, R2:{2:.7f}, MSE:{3:.7f}, RMSE:{4:.7f}".format(results.best_params_['alpha'], 
                                                                                   results.best_params_['l1_ratio'], 
                                                                                   score, mse, np.sqrt(mse)))
summary(model_best, X_scal.iloc[valid_idx], Y.iloc[valid_idx], xlabels=X.columns)

Alpha:0.0100000, l1_ratio: 0.3000000, R2:0.5171797, MSE:3169.8655476, RMSE:56.3015590
Coefficients:
              Estimate                  Std. Error         t value   p value
_intercept   45.921027  4.304046e+08+3.161352e+00j  0.0000-0.0000j  1.000000
AGE          -7.326230  2.347123e+01+7.634200e-01j -0.3118+0.0101j  0.755554
BMI         112.053983  3.218924e+01+1.973670e-01j  3.4810-0.0213j  0.000678
BP           63.835355  2.838353e+01+5.208310e-01j  2.2483-0.0413j  0.026192
S1          -10.286770  1.660045e+02+4.273140e-01j -0.0620+0.0002j  0.950683
S2          -11.664902  1.172206e+02+7.303950e-01j -0.0995+0.0006j  0.920884
S3          -51.885195  7.175728e+01+9.207300e-02j -0.7231+0.0009j  0.470919
S4           26.375923  5.750615e+01+3.575170e-01j  0.4586-0.0029j  0.647237
S5          102.402575  5.126996e+01-9.916700e-02j  1.9973+0.0039j  0.047847
S6           25.791145  3.427508e+01+6.024560e-01j  0.7522-0.0132j  0.453175
SEX_1        10.231459  4.304046e+08+5.375030e-01j  0